In [ ]:
!pip install -r requirements.txt

In [1]:
from azure.identity import AzureCliCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool
from dotenv import load_dotenv
import os

load_dotenv()

# CONNECTION_STRING = "<HostName>; <AzureSubscriptionId>; <ResourceGroup>; <HubName>"
CONNECTION_STRING=os.getenv("AZURE_CONNECTION_STRING")
SEARCH_CONNECTION_NAME = os.getenv("AI_SEARCH_CONNECTION_NAME") # Your search connection name
SEARCH_INDEX_NAME = os.getenv("AI_SEARCH_INDEX_NAME") # Your search index name

client = AIProjectClient.from_connection_string(
credential=AzureCliCredential(),
conn_str=CONNECTION_STRING,
)

conn_id = next(c.id for c in client.connections.list() if c.name == SEARCH_CONNECTION_NAME)

search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name=SEARCH_INDEX_NAME
)

agent = client.agents.create_agent(
    model="gpt-4o-mini", # Model Deployment Name
    name="my-assistant",
    instructions="You are a helpful assistant that answers questions using Azure AI Search docs",
    tools=search_tool.definitions,
    tool_resources=search_tool.resources,
)

thread = client.agents.create_thread()

client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="BQ means what in AZS?",
)

client.agents.create_and_process_run(
    thread_id=thread.id,
    assistant_id=agent.id,
)

messages = client.agents.list_messages(thread_id=thread.id)
for message in messages['data']:
    role = message['role'].upper()
    content = ""
    for content_item in message.get("content", []):
        content = content_item.get("text", {}).get("value", "")
    print(f"{role}: {content}")

ASSISTANT: In Azure AI Search (AZS), "BQ" stands for "Binary Quantization." This method compresses high-dimensional vectors by representing each component as a single bit, either 0 or 1. It significantly reduces the memory footprint and accelerates vector comparison operations, which are essential for search and retrieval tasks【3:1†source】. 

Additionally, Binary Quantization is particularly effective for embeddings that are centered around zero and for those with dimensions greater than 1024【3:1†source】【3:2†source】.
USER: BQ means what in AZS?


## OpenAI Agents SDK Example 

In [ ]:
# Azure AI Search + OpenAI Agents SDK Integration
# pip install azure-search-documents openai "openai-agents>=0.3" python-dotenv

import os
from typing import List, Optional
from pydantic import BaseModel
from dotenv import load_dotenv
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from openai import AsyncAzureOpenAI
from agents import (
    Agent, function_tool, Runner, OpenAIChatCompletionsModel,
    set_default_openai_client, set_tracing_disabled, ModelSettings
)

# Setup for notebooks
import nest_asyncio
nest_asyncio.apply()
load_dotenv()

# Initialize clients
search_client = SearchClient(
    os.environ.get("AZURE_SEARCH_SERVICE_ENDPOINT"),
    "your index-name",
    AzureKeyCredential(os.environ.get("AZURE_SEARCH_ADMIN_KEY"))
)

openai_client = AsyncAzureOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_API_VERSION", "2024-10-21"),
    azure_deployment=os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
)

# Configure SDK
set_default_openai_client(openai_client)
set_tracing_disabled(True)

# Define search tool
class SearchParams(BaseModel):
    query: str
    top: Optional[int] = 5

@function_tool
def search_azure_docs(params: SearchParams) -> List[dict]:
    """Search Azure AI Search documentation."""
    vector_query = VectorizableTextQuery(
        text=params.query,
        k_nearest_neighbors=params.top,
        fields="text_vector"
    )
    
    results = search_client.search(
        search_text=params.query,
        vector_queries=[vector_query],
        select=["chunk_id", "chunk", "title"],
        top=params.top
    )
    
    return [{
        "content": r.get("chunk", ""),
        "title": r.get("title", ""),
        "id": r.get("chunk_id", "")
    } for r in results]

# Create agent
agent = Agent(
    name="Azure AI Search Expert",
    instructions="""You are an Azure AI Search expert. Use 'search_azure_docs' tool 
    to find documentation.""",
    tools=[search_azure_docs],
    model=OpenAIChatCompletionsModel(
        model=os.environ.get("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME"),
        openai_client=openai_client
    ),
    model_settings=ModelSettings(temperature=0.3)
)

# Run agent
query = "How can I use vector search in Azure AI Search?"
result = Runner.run_sync(agent, query)
print(f"\nResponse:\n{result.final_output}")


Response:
To use vector search in Azure AI Search, you can follow these key points:

1. **Vector Storage**: Azure AI Search supports vector storage at the field level, allowing you to combine vector and non-vector fields in the same search index. You can create a vector store using the Create Index REST API or Azure SDK methods.

2. **Schema Design**: Design your schema based on your use case and intended vector retrieval pattern. Ensure to estimate index size and check service capacity.

3. **Hybrid Search**: Azure AI Search allows for hybrid scenarios where vector and keyword searches run in parallel, returning a unified result set. This often yields better results than using either method alone.

4. **Querying**: To perform a vector search, the query itself must be a vector. You can convert user input into a vector using an embedding library or API. Azure AI Search supports querying multiple vector fields and setting vector weights.

5. **Integration**: Vector search is available t